In [ ]:
## ----Importing necessary stuff----
import os  # Handling filesystems
import warnings
 
# Ignoring a silly warning from HVSRPY
warnings.simplefilter("ignore", UserWarning) 
warnings.simplefilter("ignore", RuntimeWarning) 

import numpy as np  # Numerical array management
import matplotlib.pyplot as plt  # Plot results
from matplotlib.collections import LineCollection
from matplotlib import spines
from tqdm import tqdm  # To track progress of HVSR computations
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import hann
from scipy.integrate import romb, cumulative_simpson, simpson
import pandas as pd

plt.rcParams['date.converter'] = 'concise'
plt.style.use('seaborn-v0_8-paper')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'

from obspy.clients.filesystem.sds import Client  # Locally stored data
from obspy import read, read_inventory, UTCDateTime  # Creating datetime objects

import hvsrpy  # Dear goodness
from hvsrpy import preprocess, process, TimeSeries, SeismicRecording3C, settings, sesame
from hvsrpy import frequency_domain_window_rejection as window_rejection

from IPython.display import clear_output

In [ ]:
sds = '/Users/roberto/Documents/colima-data/sds'

In [ ]:
inv = read_inventory('/Users/roberto/Documents/colima-data/response/MNGR.xml')
for canal in inv[0][0]:
    canal.location_code = '00'

In [ ]:
## ----Setting up OBSPY client----
client = Client(sds)
net = 'UC'  # Universidad de Colima
sta = 'MNGR'  # Montegrande
loc = '00'
cha = 'HH*'  # Every broadband channel

In [ ]:
st = client.get_waveforms(net, sta, loc, cha, UTCDateTime(2015,1,1), UTCDateTime(2015,1,2))
st

In [ ]:
## ----Setting up HVSRPY----
freq1 = 1
freq2 = 25
nfreqs = 200
freqs = np.geomspace(freq1,freq2,nfreqs)

# Preprocessing (detrend, window length, bandpass filter)
preproc = settings.HvsrPreProcessingSettings()  # Init 
preproc.detrend = 'linear'
preproc.window_length_in_seconds = 60  # why no overlap?
preproc.filter_corner_frequencies_in_hz = [0.1, 49.0]
print('-'*60)
print('Preprocessing Summary')
print('-'*60)
preproc.psummary()

# Processing for traditional HVSR
proc = settings.HvsrTraditionalProcessingSettings()  # Init
proc.window_type_and_width = ('tukey', 0.2)
proc.smoothing = dict(operator = 'konno_and_ohmachi',
                      bandwidth = 40,
                      center_frequencies_in_hz = freqs)
proc.method_to_combine_horizontals = 'geometric_mean'
print('-'*60)
print('Processing Summary for Traditional HVSR')
print("-"*60)
proc.psummary()

# Processing for single-azimuth HVSR (east)
proc_east = settings.HvsrTraditionalSingleAzimuthProcessingSettings()
proc_east.window_type_and_width = ('tukey', 0.2)
proc_east.smoothing = dict(operator = 'konno_and_ohmachi',
                           bandwidth = 40,
                           center_frequencies_in_hz = freqs)
proc_east.azimuth_in_degrees = 90
print('-'*60)
print('Processing Summary (East)')
print("-"*60)
proc_east.psummary()

# Window rejection
n = 2

In [ ]:
window_rejection?

In [ ]:
def HVSR_FULL_FROM_ST(stream,preproc,proc):
    '''
    Feed it a pre-merged Obspy Stream object
    for simplicity's sake
    '''
    # Load waveforms to HVSRPY
    source = SeismicRecording3C(
        TimeSeries(
            stream.select(component='N')[0].data,
            stream.select(component='N')[0].stats.delta
        ), 
        TimeSeries(
            stream.select(component='E')[0].data,
            stream.select(component='E')[0].stats.delta
        ), 
        TimeSeries(
            stream.select(component='Z')[0].data,
            stream.select(component='Z')[0].stats.delta
        )
    )
    srecords = preprocess(source,preproc)
    hvsr = process(srecords,proc)
    window_rejection(hvsr,n=n,search_range_in_hz=(1,20))
    
    return hvsr

In [ ]:
def HVSR_EAST_FROM_ST(stream, preproc, proc):
    '''
    Feed it a pre-merged Obspy Stream object
    for simplicity's sake
    '''
    # Load waveforms to HVSRPY
    source = SeismicRecording3C(
        TimeSeries(
            stream.select(component='E')[0].data,
            stream.select(component='E')[0].stats.delta
        ),
        TimeSeries(
            stream.select(component='E')[0].data,
            stream.select(component='E')[0].stats.delta
        ),
        TimeSeries(
            stream.select(component='Z')[0].data,
            stream.select(component='Z')[0].stats.delta
        )
    )
    srecords = preprocess(source, preproc)
    hvsr = process(srecords, proc)
    window_rejection(hvsr, n=n, search_range_in_hz=(1,20))

    return hvsr

In [ ]:
def datetimedayparser(utcdatetime):
    year = utcdatetime.year
    month = utcdatetime.month
    day = utcdatetime.day
    return f"{year}-{month}-{day}"

In [ ]:
ptime = UTCDateTime(2015,1,1)

totaldays = 365

days = []
dates = []
mpldates = []
curves_full = []
curves_east = []

avgamps_Z = []

peak_f0_freqs_full = []
peak_f0_amps_full = []
peak_f0_freqs_east = []
peak_f0_amps_east = []

mean_f0_freqs_full = []
mean_f0_amps_full = []
mean_f0_freqs_east = []  
mean_f0_amps_east = []  


In [ ]:
client.sds_root

In [ ]:
st = client.get_waveforms(net, sta, loc, cha, ptime, ptime+600)
st

In [ ]:
for i in tqdm(range(totaldays)):
    try:
        stime = ptime + i * 86400
        etime = stime + 86400
        day_julian = i 
        # Read waveforms
        st = client.get_waveforms(net, sta, loc, cha, stime, etime)
        st.merge(fill_value = 0)
        st.remove_sensitivity(inventory=inv)
        st.detrend('linear')
        st.filter('highpass',freq=1)
        ampdata = np.mean(np.abs(st.select(component='Z')[0].data))
        # # Load waveforms to HVSRPY
        hv_full = HVSR_FULL_FROM_ST(st,preproc,proc)
        hv_east = HVSR_EAST_FROM_ST(st,preproc,proc)

        curve_full = hv_full.mean_curve()
        curves_full.append(curve_full)
        curve_east = hv_east.mean_curve()
        curves_east.append(curve_east)


        days.append(day_julian)
        dates.append(stime)
        mpldates.append(stime.matplotlib_date)
        avgamps_Z.append(ampdata)


        peak_full_f, peak_full_a = hv_full.mean_curve_peak()
        peak_east_f, peak_east_a = hv_east.mean_curve_peak()

        peak_f0_freqs_full.append(peak_full_f)
        peak_f0_amps_full.append(peak_full_a)
        peak_f0_freqs_east.append(peak_east_f)
        peak_f0_amps_east.append(peak_east_a)

        mean_full_f = hv_full.mean_fn_frequency()
        mean_full_a = hv_full.mean_fn_amplitude()
        mean_east_f = hv_east.mean_fn_frequency()
        mean_east_a = hv_east.mean_fn_amplitude()

        mean_f0_freqs_full.append(mean_full_f)
        mean_f0_amps_full.append(mean_full_a)
        mean_f0_freqs_east.append(mean_east_f)
        mean_f0_amps_east.append(mean_east_a)

    except:
        pass
plotdates = np.array(dates).astype('datetime64[s]')

In [ ]:
d_hv = {'day':days,'date':dates,'mpldate':mpldates,'curves_east':curves_east,'curves_full':curves_full,
        'peak_east_f':peak_f0_freqs_east,'peak_east_a':peak_f0_amps_east,
        'peak_full_f':peak_f0_freqs_full,'peak_full_a':peak_f0_amps_full,
        'mean_east_f':mean_f0_freqs_east,'mean_east_a':mean_f0_amps_east,
        'mean_full_f':mean_f0_freqs_full,'mean_full_a':mean_f0_amps_full}

In [ ]:
df_hv = pd.DataFrame(data=d_hv)
df_hv.to_json('hvsr_colima_24h.json')

In [ ]:
df_hv

In [ ]:
for curve in curves_east:
    plt.plot(freqs,curve,linewidth=0.1,c='k')